<a href="https://colab.research.google.com/github/isaacdono/ml-studies/blob/main/other%20topics/self_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo Prático: Aprendizado Autossupervisionado (SSL)

O Aprendizado Autossupervisionado é uma técnica que preenche a lacuna entre o aprendizado supervisionado (que requer muitos dados rotulados) e o não supervisionado.

A ideia central é **gerar os próprios rótulos a partir dos dados não rotulados**. Para isso, criamos uma **tarefa de pretexto (pretext task)**, onde o modelo aprende a prever alguma propriedade dos próprios dados. Ao resolver essa tarefa, o modelo é forçado a aprender representações (features) ricas e úteis sobre a estrutura dos dados.

Neste notebook, vamos implementar uma forma popular de SSL chamada **Aprendizado Contrastivo**, inspirada no framework **SimCLR**.

**O Plano:**
1.  **Tarefa de Pretexto:** Pegaremos uma imagem, criaremos duas versões aumentadas (distorcidas) dela e treinaremos um modelo para "saber" que essas duas versões vieram da mesma imagem original, distinguindo-as de versões de outras imagens.
2.  **Aprendizado de Representação:** Ao fazer isso, o modelo (um Encoder) aprenderá a extrair as características essenciais de uma imagem, ignorando as variações de cor, rotação, etc.
3.  **Tarefa de Destino (Downstream Task):** Usaremos o Encoder pré-treinado para uma tarefa de classificação de imagens, usando **apenas uma pequena fração** dos rótulos. Vamos provar que essa abordagem supera em muito um modelo treinado do zero com os mesmos poucos rótulos.


In [ ]:
# Descomente e execute se não tiver a biblioteca instalada
# !pip install tensorflow_datasets

import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow versão: {tf.__version__}")

In [ ]:
# Carregando o dataset CIFAR-10
# Para o pré-treinamento SSL, vamos ignorar os rótulos.
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# --- Hiperparâmetros ---
BATCH_SIZE = 128
IMG_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Função para normalizar as imagens para o intervalo [0, 1]
def normalize_img(image, label):
    return (tf.cast(image, tf.float32) / 255.0, label)

# Pipeline de dados para o pré-treinamento (sem rótulos)
ds_train_unsupervised = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE).shuffle(1024).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Pipeline de dados para a tarefa de classificação (com rótulos)
ds_train_supervised = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_test_supervised = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

print("Dataset CIFAR-10 carregado e preparado.")


In [ ]:
"""
Esta é a parte central da nossa tarefa de pretexto. Criamos duas "visões" diferentes e aleatórias da mesma imagem.
"""

data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")


In [ ]:
def get_encoder():
    # Usando uma ResNet como nosso encoder para aprender as representações
    # include_top=False remove a camada final de classificação
    resnet = tf.keras.applications.ResNet50V2(include_top=False, weights=None, input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
    return models.Sequential([
        layers.Input((IMG_SIZE, IMG_SIZE, 3)),
        data_augmentation, # Aplicar aumento nos dados de entrada
        resnet
    ], name='encoder')


def get_ssl_model(encoder):
    # A cabeça de projeção é usada apenas durante o pré-treinamento SSL
    projection_head = models.Sequential([
        layers.Input(shape=(encoder.output.shape[1],)),
        layers.Dense(128, activation="relu"),
        layers.Dense(64)
    ], name='projection_head')

    # O modelo SSL completo
    ssl_model = models.Sequential([
        encoder,
        projection_head
    ], name='ssl_model')
    return ssl_model

encoder = get_encoder()
ssl_model = get_ssl_model(encoder)

ssl_model.summary()


In [ ]:
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, temperature=0.1, **kwargs):
        super().__init__(**kwargs)
        self.temperature = temperature

    def call(self, z1, z2):
        # z1 e z2 são as projeções das duas visões aumentadas do mesmo batch
        # O formato de z1 e z2 é [batch_size, projection_dim]

        # Normalizar as projeções
        z1 = tf.math.l2_normalize(z1, axis=1)
        z2 = tf.math.l2_normalize(z2, axis=1)

        # Matriz de similaridade de cosseno
        similarities = tf.matmul(z1, z2, transpose_b=True) / self.temperature

        # Os pares positivos estão na diagonal da matriz de similaridade entre z1 e z2
        # A perda deve maximizar a similaridade desses pares em relação a todos os outros (negativos)
        batch_size = tf.shape(z1)[0]
        labels = tf.range(batch_size)
        loss = tf.keras.losses.sparse_categorical_crossentropy(labels, similarities, from_logits=True)
        return tf.reduce_mean(loss)

In [ ]:
print("\nIniciando o pré-treinamento autossupervisionado...")

# Pré-treinamento por poucas épocas (na prática, seria por muito mais tempo)
EPOCHS_SSL = 10
optimizer = tf.keras.optimizers.Adam()
loss_fn = ContrastiveLoss()

for epoch in range(EPOCHS_SSL):
    epoch_loss = 0
    # Ignoramos os rótulos '_'
    for step, (images, _) in enumerate(ds_train_unsupervised):
        with tf.GradientTape() as tape:
            # Gerar duas visões aumentadas e suas projeções
            proj1 = ssl_model(images, training=True)
            proj2 = ssl_model(images, training=True)

            # Calcular a perda contrastiva
            loss = loss_fn(proj1, proj2)

        gradients = tape.gradient(loss, ssl_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, ssl_model.trainable_variables))
        epoch_loss += loss

    print(f"Época {epoch+1}/{EPOCHS_SSL}, Perda: {epoch_loss / (step + 1):.4f}")

print("Pré-treinamento SSL concluído!")


In [ ]:
"""
Agora, o teste final. Vamos usar nosso encoder pré-treinado e adicionar uma camada de classificação.
Vamos treiná-lo com apenas 10% dos dados rotulados e comparar seu desempenho com um modelo treinado do zero.
"""

def get_classifier(encoder, trainable=True):
    # Congelar ou não os pesos do encoder
    encoder.trainable = trainable

    model = models.Sequential([
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
        encoder,
        layers.Dense(10, activation='softmax') # 10 classes no CIFAR-10
    ], name=f'classifier_trainable_{trainable}')

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Pegar apenas 10% dos dados de treino
small_ds_train = ds_train.take(int(ds_info.splits['train'].num_examples * 0.1))
small_ds_train = small_ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# --- Modelo 1: Usando o Encoder SSL Pré-treinado ---
print("\n--- Treinando classificador com encoder SSL (congelado) ---")
classifier_ssl = get_classifier(encoder, trainable=False)
history_ssl = classifier_ssl.fit(small_ds_train, epochs=10, validation_data=ds_test_supervised, verbose=1)

# --- Modelo 2: Treinando um modelo do zero ---
print("\n--- Treinando classificador do zero ---")
encoder_scratch = get_encoder() # Novo encoder com pesos aleatórios
classifier_scratch = get_classifier(encoder_scratch, trainable=True)
history_scratch = classifier_scratch.fit(small_ds_train, epochs=10, validation_data=ds_test_supervised, verbose=1)

In [ ]:
ssl_acc = history_ssl.history['val_accuracy'][-1]
scratch_acc = history_scratch.history['val_accuracy'][-1]

print("\n--- Resultados Finais ---")
print(f"Acurácia no teste (com Encoder SSL): {ssl_acc:.4f}")
print(f"Acurácia no teste (treinado do zero): {scratch_acc:.4f}")

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_ssl.history['val_accuracy'], label='Com SSL')
plt.plot(history_scratch.history['val_accuracy'], label='Do Zero')
plt.title('Acurácia de Validação por Época')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.bar(['Com Encoder SSL', 'Do Zero'], [ssl_acc, scratch_acc], color=['cornflowerblue', 'lightcoral'])
plt.title('Acurácia Final no Conjunto de Teste')
plt.ylabel('Acurácia')
plt.ylim(0, 1)
plt.show()



In [ ]:
"""
### Conclusão

Os resultados são claros: mesmo com um pré-treinamento SSL curto e em apenas 10% dos dados rotulados, o modelo que usou o encoder pré-treinado **superou significativamente** o modelo treinado do zero.

Isso demonstra o poder do Aprendizado Autossupervisionado:
1.  **Aprende Features Úteis:** Ao resolver a tarefa de pretexto contrastiva, o encoder aprendeu a extrair características robustas das imagens, que são muito mais úteis como ponto de partida do que pesos aleatórios.
2.  **Reduz a Necessidade de Rótulos:** Conseguimos um desempenho superior com muito menos dados rotulados, um recurso caro e muitas vezes escasso.
3.  **Base para Modelos de Fundação:** Esta é a filosofia por trás de gigantescos modelos de linguagem e visão (como GPT, BERT, CLIP). Eles são pré-treinados em enormes quantidades de dados não rotulados da internet, aprendendo representações ricas que podem ser adaptadas para centenas de tarefas específicas com muito pouco ajuste fino.
"""